In [1]:
# !pip install requests tqdm 

In [2]:
import requests
import pandas as pd
import numpy as np
from pyPhenology import models
from tqdm import tqdm
import time

d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\utils.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
df = pd.read_csv('data.csv')

unnessary_features = ['Observation_ID', 'Update_Datetime', 'Observation_Time', 
                      'Site_ID', 'Site_Name', 
                      'Individual_ID', 
                      'Phenophase_ID', 'Phenophase_Category', 'Phenophase_Description', 
                      'Intensity_Category_ID', 'Intensity_Value', 'Abundance_Value', 
                      'Kingdom', 'Species_Category']

df = df.drop(columns=unnessary_features)
df = df[df['Day_of_Year'] <= 365]

# ---------- Drop Phenophase_Status == -1 and report ----------
before = len(df)
df = df[df['Phenophase_Status'] != -1].copy()
after = len(df)
print(f"Rows before: {before}, after dropping Phenophase_Status==-1: {after}, dropped: {before-after}")

# ---------- Fix -9999 error codes in climate features ----------
climate_features = ['AGDD','Tmax','Tmin','Prcp','Accum_Prcp','Daylength']
# convert -9999 -> NaN
df[climate_features] = df[climate_features].replace(-9999, np.nan)

# convert Observation_Date to datetime & create year
df['Observation_Date'] = pd.to_datetime(df['Observation_Date'])
df['year'] = df['Observation_Date'].dt.year

# create site_id from lat/lon (simple deterministic encoding)
df['site_id'] = (df['Latitude'].round(4).astype(str) + '_' + df['Longitude'].round(4).astype(str))
# Optionally map to integers:
df['site_id'] = pd.factorize(df['site_id'])[0] + 1  # site_id as ints starting at 1

# create daily mean temperature (pyPhenology examples use "temperature")
df['temperature'] = (df['Tmax'] + df['Tmin']) / 2.0

# ---------- Interpolate climate missing values per site (time-series interpolation) ----------
# sort then interpolate by site
df = df.sort_values(['site_id','Observation_Date'])
df[climate_features] = df.groupby('site_id')[climate_features].transform(
    lambda g: g.interpolate(method='linear', limit_direction='both')
)
# if still NaN (edges), fill with column median (per-site median could be used)
df[climate_features] = df[climate_features].fillna(df[climate_features].median())

# also recompute temperature if it used NaNs earlier
df['temperature'] = (df['Tmax'] + df['Tmin']) / 2.0

# ---------- Create observations DataFrame (first bloom DOY) ----------
# define first bloom per site-species-year: min Day_of_Year where Phenophase_Status==1
bloom_obs = (
    df[df['Phenophase_Status'] == 1]
    .groupby(['site_id', 'Species', 'year', 'Latitude', 'Longitude'], as_index=False)['Day_of_Year']
    .min()
    .rename(columns={'Species':'species','Day_of_Year':'doy'})
)

# If you prefer use Species_ID instead of species string:
# bloom_obs = bloom_obs.rename(columns={'Species_ID':'species_id', ...})

# pyPhenology example uses a 'phenophase' id; set a single id (e.g. 501 for 'flowers')
bloom_obs['phenophase'] = 501

# final observations DataFrame columns: ['species','site_id','year','doy','phenophase']
observations = bloom_obs[['species','site_id','year','doy','phenophase', 'Latitude', 'Longitude']]

print("Observations (first bloom) sample:")
print(observations.head())

Rows before: 6503, after dropping Phenophase_Status==-1: 6487, dropped: 16
Observations (first bloom) sample:
  species  site_id  year  doy  phenophase   Latitude   Longitude
0  annuus        1  2018   76         501  34.056213 -118.106316
1  annuus        3  2018  317         501  35.609299 -106.118263
2  annuus        4  2019  131         501  34.140465 -118.167282
3  annuus        5  2021  117         501  21.683958 -157.976212
4  annuus        5  2022    4         501  21.683958 -157.976212


In [4]:
observations.to_csv('observations.csv', index=False)

In [5]:
df.describe()

,Latitude,Longitude,Elevation_in_Meters,Species_ID,Observation_Date,Day_of_Year,Phenophase_Status,AGDD,Tmax,Tmin,Prcp,Accum_Prcp,Daylength,year,site_id,temperature
count,6487.000000,6487.000000,6487.000000,6487.000000,6487,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000,6487.000000
mean,37.529104,-122.080746,76.217049,1148.081239,2019-09-24 21:25:56.651765248,184.941730,0.156775,2905.004540,22.788048,10.319342,1.230627,343.166197,43496.177586,2019.226761,12.505164,16.553695
min,21.683958,-157.976212,0.000000,203.000000,2016-01-04 00:00:00,1.000000,0.000000,8.000000,3.000000,-9.000000,0.000000,0.000000,33523.000000,2016.000000,1.000000,-3.000000
25%,37.978359,-122.126457,63.000000,1161.000000,2017-10-03 12:00:00,97.000000,0.000000,1162.750000,17.570000,7.420000,0.000000,125.430000,37515.000000,2017.000000,12.000000,12.402500
50%,37.978359,-122.126457,63.000000,1161.000000,2019-09-26 00:00:00,185.000000,0.000000,2746.070000,22.880000,10.500000,0.000000,325.000000,43639.000000,2019.000000,12.000000,16.500000
75%,37.978359,-122.126457,63.000000,1161.000000,2021-07-24 00:00:00,273.000000,0.000000,4654.500000,27.500000,13.450000,0.000000,540.000000,49605.000000,2021.000000,12.000000,20.250000
max,43.212009,-77.429008,1872.000000,1648.000000,2023-12-31 00:00:00,365.000000,1.000000,8529.030000,42.030000,24.760000,60.000000,1584.000000,52786.000000,2023.000000,34.000000,33.125000
std,1.604425,3.431580,118.954435,133.569483,NaN,102.479229,0.363617,1883.797615,6.275225,4.160992,4.746437,248.348575,6372.269746,2.216171,2.222914,5.002215


In [ ]:
def get_nasa_temperature(lat, lon, start_year, end_year):
    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": f"{start_year}0101",
        "end": f"{end_year}1231",
        "latitude": lat,
        "longitude": lon,
        "community": "AG",
        "parameters": "T2M",
        "format": "JSON"
    }
    tries = 0
    while tries < 3:
        try:
            response = requests.get(url, params=params, timeout=30)
            data = response.json()
            temp_data = data["properties"]["parameter"]["T2M"]
            df = pd.DataFrame(list(temp_data.items()), columns=["date", "temperature"])
            df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
            df["year"] = df["date"].dt.year
            df["doy"] = df["date"].dt.dayofyear
            return df[["year", "doy", "temperature"]]
        except Exception as e:
            tries += 1
            print(f"⚠️ Retry {tries}/3 due to: {e}")
            time.sleep(2)
    return pd.DataFrame()

def build_predictor_table(obs_df):
    predictors = []
    print("🌡 Fetching temperature data for each site ...")
    for site_id, g in tqdm(obs_df.groupby('site_id')):
        lat = g['Latitude'].iloc[0]
        lon = g['Longitude'].iloc[0]
        years = g['year'].unique()
        df_temp = get_nasa_temperature(lat, lon, years.min(), years.max())
        if df_temp.empty:
            continue
        df_temp['site_id'] = site_id
        predictors.append(df_temp)
    df_pred = pd.concat(predictors, ignore_index=True)
    print(f"✅ Built predictors table: {len(df_pred)} rows.")
    return df_pred

In [9]:
predictors = build_predictor_table(observations)

🌡 Fetching temperature data for each site ...


100%|██████████| 25/25 [00:46<00:00,  1.84s/it]

✅ Built predictors table: 16799 rows.


In [15]:
predictors = predictors[predictors['doy'] <= 365].copy()

In [16]:
# print(predictors.groupby('site_id')['year'].nunique())
print(predictors.groupby('site_id')['doy'].max().describe())

count     25.0
mean     365.0
std        0.0
min      365.0
25%      365.0
50%      365.0
75%      365.0
max      365.0
Name: doy, dtype: float64


In [18]:
predictors.to_csv('predictors.csv', index=False)

In [19]:
observations = observations[['site_id', 'year', 'doy', 'phenophase']]

In [21]:
observations.head()

,site_id,year,doy,phenophase
0,1,2018,76,501
1,3,2018,317,501
2,4,2019,131,501
3,5,2021,117,501
4,5,2022,4,501


In [22]:
predictors.head()

,year,doy,temperature,site_id
0,2018,1,15.53,1
1,2018,2,17.42,1
2,2018,3,17.75,1
3,2018,4,16.86,1
4,2018,5,16.30,1


In [23]:
# ---------- Split train/test (by year) ----------
years = sorted(observations['year'].unique())
# hold out the latest year as test
test_year = years[-1]
train_years = years[:-1]

obs_train = observations[observations['year'].isin(train_years)]
obs_test  = observations[observations['year'] == test_year]

preds_train = predictors[predictors['year'].isin(train_years)]
preds_test  = predictors[predictors['year'] == test_year]

print(f"Train site-year combos: {len(obs_train)}; Test site-year combos: {len(obs_test)}")

Train site-year combos: 46; Test site-year combos: 3


In [ ]:
# ThermalTime model (standard growing-degree-days)
model = models.ThermalTime()

# Fit on training observations + predictors
# NOTE: fit may take a bit; uses scipy optimizers internally.
model.fit(obs_train, preds_train)

# fitted parameters
params = model.get_params()
print("Fitted model parameters:", params)

Fitted model parameters: {'t1': np.float64(33.0825654310663), 'T': np.float64(8.614206817541053), 'F': np.float64(991.0134587952043)}


In [25]:
y_pred = model.predict(obs_train, preds_train)
print(y_pred[:5])  # Show first 5 predictions
print("NaN ratio:", np.isnan(y_pred).mean())

[111 151 121  77  75]
NaN ratio: 0.0


In [ ]:
# ---------- Predict on test ----------
pred_vals = model.predict(obs_test, preds_test)

# convert predictions to DataFrame for merging
predictions_df = obs_test.copy()
predictions_df['doy_predicted'] = pred_vals

comparison = obs_test.merge(
    predictions_df[['site_id', 'year', 'phenophase', 'doy_predicted']],
    on=['site_id', 'year', 'phenophase'],
    suffixes=('_obs', '_pred')
)

# compute RMSE / MAE only for rows with valid predictions
from sklearn.metrics import mean_squared_error, mean_absolute_error
rmse = np.sqrt(mean_squared_error(comparison['doy'], comparison['doy_predicted']))
mae = mean_absolute_error(comparison['doy'], comparison['doy_predicted'])

print(f"✅ Evaluation complete:")
print(f"   RMSE = {rmse:.2f} days")
print(f"   MAE  = {mae:.2f} days")

print("\nSample predictions:")
print(comparison[['site_id', 'year', 'doy', 'doy_predicted']].head())


✅ Evaluation complete:
   RMSE = 47.15 days
   MAE  = 41.33 days

Sample predictions:
   site_id  year  doy  doy_predicted
0        8  2023  223            150
1       12  2023  128            149
2       13  2023  115            145


In [39]:
import joblib
joblib.dump(model, 'pyphenology_thermaltime_model.pkl')
print("Model saved to pyphenology_thermaltime_model.pkl")

Model saved to pyphenology_thermaltime_model.pkl
